In [5]:
#ignore, used to look up OG contents of input file
import os
import pandas as pd
import numpy as np
from scipy.stats import zscore

path = './data/'
filename_read = os.path.join(path,'heart.csv')
df = pd.read_csv(filename_read)


In [6]:
#Useful functions
import os
import pandas as pd

path = "./data/"

filename_read = os.path.join(path, "heart.csv")
df = pd.read_csv(filename_read)

#Randomize the data
np.random.seed(50)
df = df.reindex(np.random.permutation(df.index))
df.reset_index(inplace=True, drop=True)


headers = list(df.columns.values)
fields = []

for field in headers:
    fields.append({
        'data Val' : field,
        'mean' : df[field].mean(),
        'var' : df[field].var(),
        'sdev' : df[field].std()
    })
    
# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)

    


In [7]:
#Classification Neural Network

import tensorflow as tf 
import pandas as pd 
import numpy as np 
import requests 
import io 
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
from keras.models import Sequential 
from keras.layers.core import Dense, Activation 
from keras.callbacks import EarlyStopping

path = "./data/"

filename_read = os.path.join(path, "heart.csv") 
df = pd.read_csv(filename_read)

#one-hot coding for catagory classification data 
one_hot = pd.get_dummies(df['sex']) 
df = df.drop('sex', axis = 1) 
df = df.join(one_hot) 
df.rename(columns = { 0: "female",
                     1: "male"}, 
          inplace = True) 
one_hot = pd.get_dummies(df['fbs']) 
df = df.drop('fbs', axis = 1) 
df = df.join(one_hot)
df.rename(columns = { 0: "fbs-no",
                     1: "fbs-yes"}, 
          inplace = True) 

one_hot = pd.get_dummies(df['exang'])
df = df.drop('exang', axis = 1) 
df = df.join(one_hot)
df.rename(columns = { 0: "exang-no",
                     1: "exang-yes"},
          inplace = True)

#Normalize the real number data
df['age'] = zscore(df['age']) 
df['cp'] = zscore(df['cp']) 
df['trestbps'] = zscore(df['trestbps']) 
df['chol'] = zscore(df['chol']) 
df['restecg'] = zscore(df['restecg']) 
df['age'] = zscore(df['age']) 
df['thalach'] = zscore(df['thalach']) 
df['oldpeak'] = zscore(df['oldpeak']) 
df['slope'] = zscore(df['slope']) 
df['ca'] = zscore(df['ca']) 
df['thal'] = zscore(df['thal'])

df = df.reindex(np.random.permutation(df.index))#shuffle the data 
df.reset_index(inplace=True, drop=True)

#Pre-segment the data
mask = np.random.rand(len(df)) < 0.7
trainDF = pd.DataFrame(df[mask])
validationDF = pd.DataFrame(df[~mask])


x,y = to_xy(trainDF, "target")
x1,y1 = to_xy(validationDF, "target")

#split into train/test 
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=.30, random_state=42)#Train on 70% Tes on 30

model = Sequential()
model.add(Dense(2, input_dim=x.shape[1], activation='relu'))#Input layer
model.add(Dense(1,activation='relu')) #Hidden Layer
model.add(Dense(y.shape[1],activation='softmax')) #Output Layer
model.compile(loss='categorical_crossentropy', optimizer='adam') 
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3,patience=5, verbose=1, mode='auto')#Stop, when improvments arn't made for 5 turns              

model.fit(x,y,validation_data=(x_test,y_test),callbacks=[monitor], verbose=2,epochs=500)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 217 samples, validate on 66 samples
Epoch 1/500
 - 0s - loss: 0.6931 - val_loss: 0.6931
Epoch 2/500
 - 0s - loss: 0.6928 - val_loss: 0.6930
Epoch 3/500
 - 0s - loss: 0.6926 - val_loss: 0.6929
Epoch 4/500
 - 0s - loss: 0.6924 - val_loss: 0.6929
Epoch 5/500
 - 0s - loss: 0.6922 - val_loss: 0.6928
Epoch 6/500
 - 0s - loss: 0.6920 - val_loss: 0.6928
Epoch 00006: early stopping


In [8]:
from sklearn import metrics
import tensorflow as tf

#Create Predictions for each person in the (Prediction : Level Of Confidence)
pred = model.predict(x_test)
#print(pred)

In [9]:
#Displays the raw probability of chosen class (prediction for test set)
pred = np.argmax(pred,axis=1)
#print(pred)

In [10]:
#Calculates the percent accuacy of correct classification
y_compare = np.argmax(y_test, axis=1)
score = metrics.accuracy_score(y_compare, pred)
#print("Accuracy Score: {}".format(score))

In [11]:
#y_compare == pred

In [12]:
#Calculate log loss (Inaccuracy of predictions)
from IPython.display import display

# Dont display numpy in SI notation
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

# Generate predictions
pred = model.predict(x_test)

#print("Numpy array of predictions")
#display(pred[0]*100)

#print("As percent probability")
#display(pred[0:5]*100)

score = metrics.log_loss(y_test, pred)
#print("Log loss score: {}".format(score))